In [32]:
using DataFrames, StatFiles
using Statistics

In [12]:
# Import dataset as dataframe
df = load("NEW7080.dta") |> DataFrame;

In [22]:
indx = @. (df[:, 27] >= 30) & (df[:, 27] <= 39)
df = df[indx, :]
educ = df[:, 4]
lwklywge = df[:, 9]

329509-element Vector{Union{Missing, Float32}}:
 6.245846f0
 5.8471613f0
 6.645516f0
 6.7061334f0
 6.357876f0
 5.441835f0
 6.39066f0
 4.607667f0
 6.553961f0
 7.0170407f0
 6.39066f0
 6.9352894f0
 6.5335164f0
 ⋮
 5.9404345f0
 6.417299f0
 4.6621685f0
 5.3548613f0
 5.5987782f0
 6.2148004f0
 5.22709f0
 4.583833f0
 5.7842097f0
 5.707302f0
 5.9524937f0
 6.0477815f0

In [24]:
x = educ
y = lwklywge

329509-element Vector{Union{Missing, Float32}}:
 6.245846f0
 5.8471613f0
 6.645516f0
 6.7061334f0
 6.357876f0
 5.441835f0
 6.39066f0
 4.607667f0
 6.553961f0
 7.0170407f0
 6.39066f0
 6.9352894f0
 6.5335164f0
 ⋮
 5.9404345f0
 6.417299f0
 4.6621685f0
 5.3548613f0
 5.5987782f0
 6.2148004f0
 5.22709f0
 4.583833f0
 5.7842097f0
 5.707302f0
 5.9524937f0
 6.0477815f0

In [26]:
sort(unique(educ))

21-element Vector{Union{Missing, Int8}}:
  0
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17
 18
 19
 20

In [41]:
n = length(y)

unq_x = sort(unique(x))
px = zeros(length(unq_x))
Eyx = zeros(length(unq_x))
for u in 1:length(unq_x)
    # Estimate P(x = u) and E[y|x = u]
    indx = x .== unq_x[u]
    px[u] = sum(indx) / n
    Eyx[u] = mean(y[indx])
end
   


In [63]:
struct myIVBound
    px
    Eyx
    unq_x

    function myIVBound(y, x)
        unq_x = sort(unique(x))
        px = zeros(length(unq_x))
        Eyx = zeros(length(unq_x))
        for u in 1:length(unq_x)
            # Estimate P(x = u) and E[y|x = u]
            indx = x .== unq_x[u]
            px[u] = sum(indx) / n
            Eyx[u] = mean(y[indx])
        end
        new(px, Eyx, unq_x)
    end #MYIVBOUND
end #MYIVBOUND

In [97]:
fit = myIVBound(y, x)

myIVBound([0.0018148214464551802, 0.0006524859715516117, 0.0018390999942338448, 0.0033049173163707214, 0.004081830845287989, 0.0060787414000831545, 0.011756886761818342, 0.01874607370360142, 0.048915204137064544, 0.04000801192076696  …  0.0414981077906825, 0.37308237407779454, 0.05540668084938499, 0.07793717318798576, 0.03026624462457778, 0.10848565593049052, 0.03660901523175391, 0.035370809295042015, 0.01914060010500472, 0.03483971606238373], [5.022031784057617, 5.064202308654785, 5.16669225692749, 5.173007488250732, 5.2644267082214355, 5.330813884735107, 5.403939723968506, 5.470223426818848, 5.572460651397705, 5.646152496337891  …  5.73173189163208, 5.844363689422607, 5.917121410369873, 5.962863922119141, 6.007511138916016, 6.231886386871338, 6.229525566101074, 6.268080234527588, 6.211447715759277, 6.310892105102539], Union{Missing, Int8}[0, 1, 2, 3, 4, 5, 6, 7, 8, 9  …  11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [110]:
function coef(fit::myIVBound, s, t)
    Eyx_px = fit.Eyx .* fit.px

    from_t = fit.unq_x .> t
    until_s = fit.unq_x .< s
    
    ub = sum(Eyx_px[from_t]) + (fit.Eyx[fit.unq_x .== t] * sum(fit.px[.!from_t]))[1]
    lb = sum(Eyx_px[until_s]) + (fit.Eyx[fit.unq_x .== s] * sum(fit.px[.!until_s]))[1]
    
    return ub - lb
end

coef (generic function with 1 method)

In [114]:
coef(fit, 12, 16)/4

0.11202714900306754

In [112]:
bounds = zeros(19)
for j in 2:20
    bounds[j - 1] = coef(fit, j - 1, j)
end

In [113]:
bounds

19-element Vector{Float64}:
 0.8361493989101385
 0.7339395208060582
 0.728347306811429
 0.6384001837934452
 0.5740888677438534
 0.5042198905181223
 0.4448289522360005
 0.35468933004610737
 0.2935308064467135
 0.2684520422704866
 0.2564983532992233
 0.21343974487222006
 0.21454698115353832
 0.23170190633874288
 0.3916546646499208
 0.3369873794462892
 0.37239451417343794
 0.3153741336578584
 0.4144109499808719

In [103]:
Eyx_px = fit.Eyx .* fit.px

from_t = fit.unq_x .> t
until_s = fit.unq_x .< s
from_s = fit.unq_x .> s

21-element BitVector:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 1
 1

In [101]:
ub = sum(Eyx_px[from_t]) + (fit.Eyx[fit.unq_x .== t] * sum(fit.px[.!from_t]))[1]
lb = sum(Eyx_px[until_s]) + (fit.Eyx[fit.unq_x .== s] * sum(fit.px[from_s]))[1]

3.606903817354834

In [102]:
ub - lb

2.6285376762361152